# Self-Discover Framework Implementation

This notebook implements the Self-Discover framework for enabling LLMs to autonomously discover reasoning structures for complex tasks, as described in the research paper.

The framework consists of two main stages:
1. Discover an intrinsic reasoning structure for the task 
2. Use the discovered structure to solve problem instances

Stage 1 has three actions to compose the reasoning structure:
1. SELECT relevant reasoning modules
2. ADAPT selected modules to be task-specific
3. IMPLEMENT adapted modules into an explicit structure

Stage 2 simply follows the reasoning structure to solve each problem instance.

In [ ]:
import openai

def Self_Discover(task, reasoning_modules):
    """Main Self-Discover framework
    
    Args:
        task (str): The task to solve
        reasoning_modules (list): List of available reasoning modules
        
    Returns:
        list: Solutions to the task instances
    """
    
    # Stage 1: Discover Reasoning Structure on Task-Level
    
    # SELECT relevant reasoning modules
    selected_modules = SELECT(reasoning_modules, task_examples) 
    
    # ADAPT selected modules to be task-specific
    adapted_modules = ADAPT(selected_modules, task_examples)
    
    # IMPLEMENT adapted modules into a reasoning structure
    reasoning_structure = IMPLEMENT(adapted_modules, task_examples)
    
    # Stage 2: Solve Problems Using Discovered Structure on Instance-Level
    solutions = []
    for problem_instance in task:
        # Follow reasoning structure to solve problem
        solution = SOLVE(problem_instance, reasoning_structure)
        solutions.append(solution)
        
    return solutions

In [ ]:
def SELECT(reasoning_modules, task_examples):
    """Use LLM to select subset of relevant reasoning modules
    
    Args:
        reasoning_modules (list): List of available reasoning modules 
        task_examples (list): Examples of the task
        
    Returns:
        list: Selected subset of reasoning modules
    """
    
    # Generate prompt for LLM to select modules
    prompt = generate_select_prompt(reasoning_modules, task_examples)
    
    # Query LLM to select relevant modules 
    selected_modules = openai.Completion.create(
        engine="text-davinci-002",
        prompt=prompt,
        max_tokens=100
    )
    
    return selected_modules.choices[0].text.strip().split(',')

In [ ]:
def ADAPT(selected_modules, task_examples):
    """Use LLM to rephrase selected modules to be task-specific
    
    Args:
        selected_modules (list): Subset of reasoning modules
        task_examples (list): Examples of the task
        
    Returns:
        list: Adapted task-specific module descriptions  
    """
    
    # Generate prompt for LLM to adapt modules
    prompt = generate_adapt_prompt(selected_modules, task_examples)
    
    # Query LLM to rephrase modules to be task-specific
    adapted_modules = openai.Completion.create(
        engine="text-davinci-002",
        prompt=prompt,
        max_tokens=200
    )
    
    return adapted_modules.choices[0].text.strip().split('\n')

In [ ]:
def IMPLEMENT(adapted_modules, task_examples):
    """Use LLM to implement adapted modules into a reasoning structure
    
    Args:
        adapted_modules (list): Task-specific module descriptions
        task_examples (list): Examples of the task
        
    Returns:
        str: JSON string representing the reasoning structure
    """
    
    # Retrieve example of human-written reasoning structure 
    human_example_structure = retrieve_human_example_structure()
    
    # Generate prompt for LLM to implement structure
    prompt = generate_implement_prompt(human_example_structure, 
                                       adapted_modules, 
                                       task_examples)
    
    # Query LLM to generate reasoning structure
    reasoning_structure = openai.Completion.create(
        engine="text-davinci-002",
        prompt=prompt,
        max_tokens=500  
    )
    
    return reasoning_structure.choices[0].text.strip()

In [ ]:
def SOLVE(problem_instance, reasoning_structure):
    """Use LLM to follow reasoning structure and solve problem instance
    
    Args:
        problem_instance (str): A single instance of the task 
        reasoning_structure (str): JSON string of reasoning structure
        
    Returns:
        str: The solution to the problem instance
    """
    
    # Generate prompt for LLM to solve instance
    prompt = generate_solve_prompt(problem_instance, reasoning_structure)
    
    # Query LLM to follow structure and solve
    solution = openai.Completion.create(
        engine="text-davinci-002",
        prompt=prompt,
        max_tokens=200
    )
    
    return solution.choices[0].text.strip()

## Example Usage

Here's an example of using the Self-Discover framework to solve a complex reasoning task.

In [ ]:
task = [
    "Instance 1 of the complex task",
    "Instance 2 of the complex task"
]

reasoning_modules = [
    "break problem into steps",
    "use logical reasoning", 
    "leverage world knowledge",
    "think creatively"
]

solutions = Self_Discover(task, reasoning_modules)

print(solutions)